In [1]:
from read_data import read_data


def read_data_wrapper(setname, type):
    # data = pd.read_hdf(
    #     f"{original_data_path}/{type}{setname}.zarr"
    # )  # fill in your path and file name
    data = read_data(setname, type)
    return data[["case_id", "time", "x_leader", "x_follower", "v_leader", "v_follower"]]



In [2]:
data_HA = read_data_wrapper("HA", "train")
data_HA.head(10)

,case_id,time,x_leader,x_follower,v_leader,v_follower
0,0,0.0,0.000000,-9.428864,2.629775,0.784441
1,0,0.1,0.263154,-9.341833,2.665530,0.810204
2,0,0.2,0.535099,-9.265412,2.776808,0.828016
3,0,0.3,0.828920,-9.107064,2.966034,0.889182
4,0,0.4,1.147062,-8.933068,3.168645,0.979196
5,0,0.5,1.486376,-8.735666,3.348807,1.080297
6,0,0.6,1.844605,-8.579100,3.544290,1.174214
7,0,0.7,2.226670,-8.416525,3.764669,1.322533
8,0,0.8,2.631701,-8.234040,3.978872,1.476311
9,0,0.9,3.056757,-7.929120,4.142739,1.697523


In [6]:
import numpy as np
import pandas as pd


from tqdm import tqdm




def create_sequences(data, n_steps_in, n_steps_out, step):
    X, y = [], []
    for i in range(0, len(data) - n_steps_in - n_steps_out + 5, step):
        seq_x = data[i : i + n_steps_in]
        seq_y = data[i + n_steps_in : i + n_steps_in + n_steps_out]
        if seq_x.shape[0] == n_steps_in and seq_y.shape[0] == n_steps_out:
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)


data = data_HA.copy()

data["delta_velocity"] = data["v_follower"] - data["v_leader"]
data["delta_position"] = data["x_leader"] - data["x_follower"]
data = data[["case_id", "time", "delta_position", "delta_velocity", "v_follower"]]
data = data.sort_values(by=["case_id", "time"]).set_index("case_id")

# data2 = data2.reset_index(drop=True)
features = []
labels = []
data_prescale = []
X = []
y = []

data.head()
for case_id in tqdm(data.index.unique()):
    df = data.loc[case_id][["delta_velocity", "delta_position", "v_follower"]]
    # print("df is")
    # print(df.head(3))
    X_seqs, y_seqs = create_sequences(df, 30, 10, 1)
    X.append(X_seqs)
    y.append(y_seqs)
    # features.append(
    #     (
    #         history[["time", "delta_velocity", "delta_position", "v_follower"]],
    #         future[["time", "delta_velocity", "delta_position", "v_follower"]],
    #     )
    # )
    # labels.append(
    #     (
    #         history[["time", "v_follower"]],
    #         future[["time", "v_follower"]],
    #     )
    # )

100%|██████████| 26394/26394 [04:00<00:00, 109.91it/s]


In [9]:
from sklearn.preprocessing import StandardScaler
import pickle

# X_out = np.array(X)
# y_out = np.array(y)
# features = pd.concat(features).reset_index()
# Standardize features

# scaler must be
scaler = StandardScaler()
features = scaler.fit_transform(X, y)

features.to_hdf("../ready_data/" + "HA.h5")

with open("../ready_data/HA_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# read scaler

# with open("file/path/scaler.pkl", "rb") as f:
#     sc = pickle.load(f)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (26394,) + inhomogeneous part.